# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/Users/ammirabile/Desktop/Projects/Udacity/Udacity-Azure-Data-Engineer-Nanodegree/P1_ETLPipelines/workspace/home


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)
from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [6]:

try:
    session.execute("""
        CREATE KEYSPACE IF NOT EXISTS udacity
        WITH REPLICATION = 
        {'class' : 'SimpleStrategy', 'replication_factor' : '1'}""")
    
except Exception as e:
    print(e) 


#### Set Keyspace

In [7]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)


### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




The next blocks of code are creating the table song_info_by_session_info for question 1 and also queries the table as per the requirement
The table requires a composite key with  sessionId and itemInSession to enable filtering

In [8]:
#['artist','firstName','gender','itemInSession','lastName','length','level','location','sessionId','song','userId']

try:
    session.execute("create table if not exists song_info_by_session_info (sessionId int, itemInSession int,artist text, song text,length float, PRIMARY KEY (sessionId, itemInSession))")
except Exception as e:
    print(e)


                    

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_info_by_session_info (sessionId,itemInSession,artist,song,length) "
        query = query + "VALUES (%s,%s,%s,%s,%s)"
        session.execute(query, (int(line[8]), int(line[3]),line[0],line[1],float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
query1 = "select artist, song, length from song_info_by_session_info where sessionId = 338 and itemInSession = 4"

try:
    df = pd.DataFrame(data = session.execute(query1))
    print(df)
except Exception as e:
    print(e)

      artist song      length
0  Faithless  Ava  495.307312


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

The query below is creating the table session_song_info_by_user for question 2 and also queries the table as per the requirement
The table requires a composite key with userId and sessionId to enable the filtering and itemInSession as clustering column to sort the data alphabetically

In [11]:
#['artist','firstName','gender','itemInSession','lastName','length','level','location','sessionId','song','userId']


query2 = "select artist, song, itemInSession, firstName, lastName from session_song_info_by_user where userId = 10 and sessionId = 182"

try:
    session.execute("""create table if not exists session_song_info_by_user
    (userId int, sessionId int,itemInSession text, artist text,song text,firstName text,lastName text, 
    PRIMARY KEY ((userId, sessionId), itemInSession ))""")
except Exception as e:
    print(e)


with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO session_song_info_by_user (userId,sessionId,itemInSession,artist,song,firstName,lastName) "
        query = query + "VALUES (%s,%s,%s,%s,%s,%s,%s)"
        session.execute(query, (int(line[10]), int(line[8]),line[3],line[0],line[9],line[1],line[4]))   
        
try:
    rows = session.execute(query2)
    for row in rows:
        print(row)
except Exception as e:
    print(e)
        
        

Row(artist='Down To The Bone', song="Keep On Keepin' On", iteminsession='0', firstname='Sylvie', lastname='Cruz')
Row(artist='Three Drives', song='Greece 2000', iteminsession='1', firstname='Sylvie', lastname='Cruz')
Row(artist='Sebastien Tellier', song='Kilometer', iteminsession='2', firstname='Sylvie', lastname='Cruz')
Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', iteminsession='3', firstname='Sylvie', lastname='Cruz')


The query below is creating the table for question 3 and also queries the table as per the requirement
The table requires a composite key with the columns song, to enable the filtering, and userID to ensure users where the first name and last name are the same don't get missed during the creation of the table

In [12]:
#['artist','firstName','gender','itemInSession','lastName','length','level','location','sessionId','song','userId']
query3 = "select firstName, lastName from user_infor_by_song where song = 'All Hands Against His Own'"


try:
    session.execute("""create table if not exists user_infor_by_song
    (song text,firstName text, lastName text, userId int,
    PRIMARY KEY (song,userId))""")
except Exception as e:
    print(e)


with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_infor_by_song (song,firstName,lastName,userId) "
        query = query + "VALUES (%s,%s,%s,%s)"
        session.execute(query, (line[9], line[1],line[4],int(line[10]))) 
        
try:
    rows = session.execute(query3)
    for row in rows:
        print(row)
except Exception as e:
    print(e)
        
                    

Row(firstname='Jacqueline', lastname='Lynch')
Row(firstname='Tegan', lastname='Levine')
Row(firstname='Sara', lastname='Johnson')


### Drop the tables before closing out the sessions

In [4]:
## TO-DO: Drop the table before closing out the sessions

In [13]:
def dropTable(table):
    session.execute("drop table " + table)
    
try:
    dropTable("user_infor_by_song")
    dropTable("song_info_by_session_info")
    dropTable("session_song_info_by_user")
except Exception as e:
    print(e)
    

    

### Close the session and cluster connection¶

In [14]:
session.shutdown()
cluster.shutdown()